In [186]:
import pandas as pd
import matplotlib.pyplot as plt 
from scipy.stats import wilcoxon
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon


# Podejście 2

In [187]:
#Wczytywanie plików i liczenie średnich
def wczytaj_i_srednia(path):
    meta_cols = ["Module_Colors", "Locus_Tag", "Genes", "PGFam"]
    # Wczytanie pliku CSV
    df = pd.read_csv(path)
    
    # kolumny z wartościami liczbowymi
    value_cols = [col for col in df.columns if col not in meta_cols]

    # funkcja wydobywająca nazwę grupy z kolumny
    def get_group(col):
        return col.rsplit("_", 1)[0]

    # mapowanie kolumna → grupa
    groups = {col: get_group(col) for col in value_cols}

    # obliczanie średnich
    group_means = pd.DataFrame()

    for group in sorted(set(groups.values())):
        group_cols = [col for col in value_cols if groups[col] == group]
        group_means[group + "_mean"] = df[group_cols].mean(axis=1)

    # zwrot metadanych + średnich
    return pd.concat([df[meta_cols], group_means], axis=1)

In [188]:
MRSA_mean=wczytaj_i_srednia("MRSA.csv")
MSSA_mean=wczytaj_i_srednia("MSSA.csv")
SEPI_mean=wczytaj_i_srednia("SEPI.csv")

In [189]:
def wilcoxon_test(df, vic_col="MRSA_Vic_mean", alpha=0.01):
    """
    Wektorowa wersja testu Wilcoxona dla każego wiersza.
    Zwraca DataFrame z p_value, p_adj, significant, rest_mean, diff.
    """

    meta_cols = ["Module_Colors", "Locus_Tag", "Genes", "PGFam"]
    mean_cols = [c for c in df.columns if c.endswith("_mean") and c not in meta_cols]
    df[mean_cols] = df[mean_cols].apply(pd.to_numeric, errors="coerce")

    other_cols = [c for c in mean_cols if c != vic_col]

    rest_vals = df[other_cols].to_numpy(dtype=float)
    vic_vals = df[vic_col].to_numpy(dtype=float)

    rest_mean = np.nanmean(rest_vals, axis=1)
    diff = vic_vals - rest_mean

    p_values = np.full(df.shape[0], np.nan)

    # Wilcoxon dla każdego wiersza
    for i in range(df.shape[0]):
        row = rest_vals[i, :]
        vic = vic_vals[i]
        row_clean = row[~np.isnan(row)]
        if len(row_clean) >= 1 and not np.isnan(vic):
            try:
                stat, p_values[i] = wilcoxon(row_clean - vic)
            except ValueError:
                p_values[i] = np.nan

    # korekcja FDR (Benjamini-Hochberg)
    valid = ~np.isnan(p_values)
    m = valid.sum()
    p_adj = np.full_like(p_values, np.nan)
    significant = np.full_like(p_values, False, dtype=bool)

    if m > 0:
        sorted_idx = np.argsort(p_values[valid])
        sorted_p = p_values[valid][sorted_idx]
        adjusted = sorted_p * m / (np.arange(1, m+1))
        adjusted = np.minimum.accumulate(adjusted[::-1])[::-1]
        adjusted = np.clip(adjusted, 0, 1)
        p_adj[valid] = adjusted[np.argsort(sorted_idx)]
        significant = p_adj < alpha

    results_df = pd.DataFrame({
        "p_value": p_values,
        "p_adj": p_adj,
        "significant": significant,
        "rest_mean": rest_mean,
        "diff": diff
    })

    final_df = pd.concat([df.reset_index(drop=True), results_df], axis=1)

    return final_df


In [196]:
MRSA_result_wilcoxon= wilcoxon_test(MRSA_mean, vic_col="MRSA_Vic_mean")

In [197]:
MSSA_result_wilcoxon= wilcoxon_test(MSSA_mean, vic_col="MSSA_Vic_mean")

In [198]:
SEPI_result_wilcoxon= wilcoxon_test(SEPI_mean, vic_col="SEPI_Vic_mean")

In [199]:
# df = wynik funkcji test_vic_significance lub wersji Wilcoxona
def top_genes(df, top_n=5):
    # filtrujemy tylko geny istotne po korekcji FDR
    significant_df = df[df['p_adj'] < 0.05]
    
    # obliczamy absolutną różnicę dla rankingu
    significant_df["abs_diff"] = significant_df["diff"].abs()
    
    # sortujemy malejąco wg największej różnicy
    top_genes_df = significant_df.sort_values("abs_diff", ascending=False)
    
    # dodajemy kolumnę VIC_mean dla porządku
    top_genes_df["VIC_mean"] = top_genes_df["diff"] + top_genes_df["rest_mean"]
    
    # wybieramy kolumny do zwrócenia
    top_genes_df = top_genes_df[["Genes", "Locus_Tag", "PGFam", "diff", "rest_mean", "VIC_mean"]]
    
    return top_genes_df.head(top_n)


In [200]:
MRSA_top = top_genes(MRSA_result_wilcoxon)
MSSA_top = top_genes(MSSA_result_wilcoxon)
SEPI_top = top_genes(SEPI_result_wilcoxon)


/tmp/ipykernel_11579/3535897921.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  significant_df["abs_diff"] = significant_df["diff"].abs()
/tmp/ipykernel_11579/3535897921.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  significant_df["abs_diff"] = significant_df["diff"].abs()
/tmp/ipykernel_11579/3535897921.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [201]:
MRSA_top.to_csv("MRSA_top_gene.csv")
MSSA_top.to_csv("MSSA_top_gene.csv")
SEPI_top.to_csv("SEPI_top_gene.csv")

# Podejście 2

In [202]:
def analyze_file_simple(path, vic_threshold=1.5, diff_threshold=2.75):
    df = pd.read_csv(path)

    #wykrywanie prefixu w kolumnach (MRSA/MSSA/SEPI)
    example = [c for c in df.columns if "_As_" in c][0]
    prefix = example.split("_")[0]

    # kolumny dla Vic i pozostałych warunków
    vic_cols = [c for c in df.columns if f"{prefix}_Vic" in c]
    other_cols = [c for c in df.columns if c.startswith(prefix + "_") and "Vic" not in c]

    # filtrujemy geny spełniające kryteria
    vic_genes = set()
    for _, row in df.iterrows():
        vic_mean = row[vic_cols].mean()
        other_mean = row[other_cols].mean()
        if vic_mean > vic_threshold and (vic_mean - other_mean) > diff_threshold:
            vic_genes.add(str(row["Locus_Tag"]))

    return vic_genes


In [203]:
def analyze_file(path, vic_threshold=1.5, diff_threshold=2.75):
    # Wczytanie pliku CSV
    df = pd.read_csv(path)

    # Wykrywanie prefixu w kolumnach (MRSA/MSSA/SEPI)
    example = [c for c in df.columns if "_As_" in c][0]
    prefix = example.split("_")[0]

    # Kolumny dla wirulencji (Vic)
    vic_cols = [f"{prefix}_Vic_1",
                f"{prefix}_Vic_2",
                f"{prefix}_Vic_3"]

    # Wszystkie inne kolumny poza Vic
    other_cols = [c for c in df.columns if c.startswith(prefix + "_") and "Vic" not in c]

    # Obliczanie średniej dla Vic i średniej dla wszystkich innych kolumn
    df["Vic_mean"] = df[vic_cols].mean(axis=1)
    df["Other_mean"] = df[other_cols].mean(axis=1)
    df["Diff"] = df["Vic_mean"] - df["Other_mean"]

    # Filtrowanie genów spełniających kryteria
    vic_specific = df[(df["Vic_mean"] > vic_threshold) & (df["Diff"] > diff_threshold)]

    # Zwrócenie listy genów jako zbiór
    return set(vic_specific["Locus_Tag"].astype(str))


In [204]:
genes_1 = analyze_file("MRSA.csv")
genes_2 = analyze_file("MSSA.csv")
genes_3 = analyze_file("SEPI.csv")
#genes_3_5 = analyze_file("SEPI.csv", 1.5, 2.5)
print(genes_1)
print(genes_2)
print(genes_3)
#print(genes_3_75)

{'SAR_RS00735', 'SAR_RS05625', 'SAR_RS02160', 'SAR_RS05640', 'SAR_RS05650', 'SAR_RS05655', 'SAR_RS05645', 'SAR_RS09090', 'SAR_RS06000', 'SAR_RS02165'}
{'SAS_RS03990', 'SAS_RS00635', 'SAS_RS02055', 'SAS_RS00025', 'SAS_RS13000', 'SAS_RS08630', 'SAS_RS08635', 'SAS_RS02060', 'SAS_RS03980', 'SAS_RS13210'}
{'B4U56_RS02555', 'B4U56_RS02570'}


In [205]:
genes_1 = analyze_file_simple("MRSA.csv")
genes_2 = analyze_file_simple("MSSA.csv")
genes_3 = analyze_file_simple("SEPI.csv")
#genes_3_5 = analyze_file("SEPI.csv", 1.5, 2.5)
print(genes_1)
print(genes_2)
print(genes_3)

{'SAR_RS00735', 'SAR_RS05625', 'SAR_RS02160', 'SAR_RS05640', 'SAR_RS05650', 'SAR_RS05655', 'SAR_RS05645', 'SAR_RS09090', 'SAR_RS06000', 'SAR_RS02165'}
{'SAS_RS03990', 'SAS_RS00635', 'SAS_RS02055', 'SAS_RS00025', 'SAS_RS13000', 'SAS_RS08630', 'SAS_RS08635', 'SAS_RS02060', 'SAS_RS03980', 'SAS_RS13210'}
{'B4U56_RS02555', 'B4U56_RS02570'}


In [206]:
# Funkcja do zapisywania zbioru genów do pliku CSV
def save_genes_to_csv(genes_set, filename):
    # Konwertujemy set na listę i tworzymy DataFrame
    df = pd.DataFrame(list(genes_set), columns=["Locus_Tag"])
    # Zapis do CSV
    df.to_csv(filename, index=False)


In [207]:
save_genes_to_csv(genes_1, "MRSA_genes.csv")
save_genes_to_csv(genes_2, "MSSA_genes.csv")
save_genes_to_csv(genes_3, "SEPI_genes.csv")
